In [113]:
import torch
import random

In [123]:
# 设置原始数据
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.1, y.shape)
    return X, y.reshape(-1, 1)
true_w = torch.tensor([3, -2.5])
true_b = 3.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [115]:
# 提取小批量数据
def data_iter(batch_size, features, labels):
    num = len(features)
    indices = list(range(num))
    random.shuffle(indices)
    for i in range(0, num, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num)])
        yield features[batch_indices], labels[batch_indices]

In [116]:
size = 10
for X, y in data_iter(size, features, labels):
    print(X, '\n', y)
    break

tensor([[-0.1431, -0.7597],
        [-0.8058, -0.8069],
        [-1.7128,  1.1963],
        [ 0.5841,  1.0643],
        [ 0.1308,  1.4808],
        [-0.2017, -1.4701],
        [ 2.2148,  0.2915],
        [-0.2198, -1.1168],
        [-1.2746,  1.1294],
        [-0.5958,  0.0961]]) 
 tensor([[ 4.7067],
        [ 2.7544],
        [-4.9427],
        [ 2.4115],
        [-0.1670],
        [ 6.2370],
        [ 9.0719],
        [ 5.1519],
        [-3.4829],
        [ 0.9781]])


In [117]:
# 设定模型与损失函数
def Linear(X, w, b):
    return torch.matmul(X, w) + b
def loss_squared(y_hat, y_true):
    return (y_hat - y_true.reshape(y_hat.shape)) ** 2 / 2

In [118]:
# 设定优化算法
def sgd(params, learn_rate, batch_size):
    with torch.no_grad():
        for param in params:
            param -= learn_rate * param.grad / batch_size
            param.grad.zero_()

In [119]:
# 初始化参数
w = torch.normal(0, 1, size=(2,), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [120]:
# 设置超参数
learn_rate = 0.03
num_eqochs = 3
batch_size = 10
net = Linear
loss = loss_squared

In [121]:
# 训练
for eqoch in range(num_eqochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], learn_rate, batch_size)

In [122]:
print(true_w - w.reshape(true_w.shape))
print(true_b - b)

tensor([-0.0002,  0.0018], grad_fn=<SubBackward0>)
tensor([-0.0017], grad_fn=<RsubBackward1>)
